In [39]:
my_key = "afc7201593c1f2fc08c24eec64fabc88"
api_request = "https://api.themoviedb.org/3/movie/550?api_key=afc7201593c1f2fc08c24eec64fabc88"

In [30]:
#import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import seaborn as sns

In [31]:
#iterate through 499 API requests. Each "page" contains 20 records. Records are stored in 'collected_data'
#this cell takes approximately 5 minutes to run

collected_data = []

for i in range(1,500,1):
    request_string = "https://api.themoviedb.org/3/discover/movie?api_key=afc7201593c1f2fc08c24eec64fabc88&language=en-US&sort_by=revenue.desc&include_adult=false&include_video=false&primary_release_date.lte=2019-12-30&primary_release_date.gte=2000-01-01&page="+str(i)
    data = requests.get(request_string)
    data = data.json()['results']
    for item in data:
        collected_data.append(item)

In [32]:
#load data into dataframe
df = pd.DataFrame(collected_data)

In [35]:
#open dataframe to review returned columns.
df.head()

,popularity,vote_count,video,poster_path,id,adult,backdrop_path,original_language,original_title,genre_ids,title,vote_average,overview,release_date
0,40.636,14611,False,/or06FN3Dka5tukK1e9sl16pB3iy.jpg,299534,False,/orjiB3oUIsyz60hoEqkiGpy5CeO.jpg,en,Avengers: Endgame,"[28, 12, 878]",Avengers: Endgame,8.3,After the devastating events of Avengers: Infi...,2019-04-24
1,36.426,21849,False,/6EiRUJpuoeQPghrs3YNktfnqOVh.jpg,19995,False,/bIL7ENqh1egWTxN41sM2W42JqPc.jpg,en,Avatar,"[28, 12, 14, 878]",Avatar,7.4,"In the 22nd century, a paraplegic Marine is di...",2009-12-10
2,31.247,14812,False,/wqnLdwVXoBjKibFRR5U3y0aDUhs.jpg,140607,False,/k6EOrckWFuz7I4z4wiRwz8zsj4H.jpg,en,Star Wars: The Force Awakens,"[28, 12, 14, 878]",Star Wars: The Force Awakens,7.4,Thirty years after defeating the Galactic Empi...,2015-12-15
3,82.930,19327,False,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,299536,False,/bOGkgRGdhrBYJSLpXaxhXVstddV.jpg,en,Avengers: Infinity War,"[28, 12, 878]",Avengers: Infinity War,8.3,As the Avengers and their allies have continue...,2018-04-25
4,30.958,15896,False,/rhr4y79GpxQF9IsfJItRXVaoGs4.jpg,135397,False,/yOCRqvrRrxbs5FYq2pX1KtLJwmR.jpg,en,Jurassic World,"[28, 12, 878, 53]",Jurassic World,6.6,Twenty-two years after the events of Jurassic ...,2015-06-06


In [36]:
#using the 'discover' API is great for pulling summary data for a list of movies.
#however, it does not pull all the information that TMDB has on any particular movie
#primarily, the 'discover' API is missing budget, revenue, imdb_id, and runtime
#these can be pulled from the 'movie' API call. this cell prepares for the next for loop to pull this required data
budget = []
imdb_id = []
revenue = []
runtime = []
production_companies = []
production_countries = []
genres = []

In [37]:
#create a list of tmdb-movie-ids to iterate through. each id will be used in its own API call
movie_ids = list(df['id'])

In [38]:
#for each movie_id defined above, call API to to retrieve budget, revenue, imdb_id, runtime.
#tmdb data structure isn't uniform. movie dictionaries do not necessarily have all keys.
#to get around this, we used try/except
#this cell takes about 30 minutes to run

for movie_id in movie_ids:
    request_string = "https://api.themoviedb.org/3/movie/"+str(movie_id)+"?api_key=afc7201593c1f2fc08c24eec64fabc88&language=en-US"
    data = requests.get(request_string)
    data = data.json()
    try:
        budget.append(data['budget'])
    except:
        budget.append(" ")
    try:
        imdb_id.append(data['imdb_id'])
    except:
        imdb_id.append(" ")
    try:
        revenue.append(data['revenue'])
    except:
        revenue.append(" ")
    try:
        runtime.append(data['runtime'])
    except:
        runtime.append(" ")
    try:
        production_companies.append(data['production_companies'])
    except:
        production_companies.append(" ")
    try:
        production_countries.append(data['production_countries'])
    except:
        production_countries.append(" ")
    try:
        genres.append(data['genres'])
    except:
        genres.append(" ")

In [39]:
#set new df columns equal to the lists created above.
df['budget'] = budget
df['imdb_id'] = imdb_id
df['revenue'] = revenue
df['runtime'] = runtime
df['production_companies'] = production_companies
df['production_countries'] = production_countries
df['genres'] = genres

In [52]:
to_drop = df[(df['budget']==0) | (df['revenue']==0)].index
new_df = df.drop(to_drop)

In [55]:
to_drop = new_df[(new_df['revenue']==" ") | (new_df['revenue']=="") | (new_df['revenue']=="nan")].index
new_df.drop(to_drop, inplace = True)

In [60]:
#drop columns = ['poster_path' and 'backdrop_path']
new_df.drop(['poster_path','backdrop_path'], axis = 1, inplace = True)

In [71]:
#convert revenue to integer
new_df['revenue'] = new_df['revenue'].astype("int64")

In [72]:
#convert budget to integer
new_df['budget'] = new_df['budget'].astype("int64")

In [79]:
#save dataframe as csv
new_df.to_csv('tmdb_data')